In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from sklearn import metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import random
import warnings

# Suppressing warnings
warnings.filterwarnings("ignore")

In [ ]:
# Loading Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

# Reshaping image data to match CNN input shape
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

# Converting labels to categorical format
train_labels_categorical = tf.keras.utils.to_categorical(train_labels)
test_labels_categorical = tf.keras.utils.to_categorical(test_labels)

In [ ]:
# Storing the length of training data
train_data_length = len(train_images)

# Storing the shape of test labels
test_labels_shape = test_labels.shape

In [ ]:
# Normalizing pixel values to the range [0, 1]
train_images_normalized = train_images.astype('float32') / 255.0
test_images_normalized = test_images.astype('float32') / 255.0


In [ ]:
# Class names for visualization
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
# Visualizing sample images from the dataset
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [ ]:
# Defining the CNN model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu', padding='same', name='conv-layer-1'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2), name='pooling-layer-1'),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', name='conv-layer-2'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2), name='pooling-layer-2'),
    tf.keras.layers.GlobalAveragePooling2D(name='pooling-layer-3'),
    tf.keras.layers.Dense(len(class_names), activation="softmax", name="output-layer")
])

In [ ]:
# Compiling the model
model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

In [ ]:
# Training the model
history = model.fit(train_images, train_labels_categorical, epochs=10, validation_data=(test_images, test_labels_categorical))

In [ ]:
# Displaying model summary
model.summary()

In [ ]:
# Plotting training history (loss and accuracy)
pd.DataFrame(history.history).plot(figsize=(10, 7))
plt.title("Metrics Graph")
plt.show()

In [ ]:
# Evaluating the model on the test data
model.evaluate(test_images, test_labels_categorical)

In [ ]:
# Making predictions on test data
predictions = model.predict(test_images)

In [ ]:
# Converting predictions and true labels to numerical format
predictions = tf.argmax(predictions, axis=1)
true_labels = tf.argmax(test_labels_categorical, axis=1)

In [ ]:
# Computing and printing accuracy score
accuracy = metrics.accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)


In [ ]:
# Generating classification report
classification_report = metrics.classification_report(true_labels, predictions)
print("Classification Report:\n", classification_report)

In [ ]:
# Generating and plotting confusion matrix
confusion_matrix = metrics.confusion_matrix(true_labels, predictions)
plot_confusion_matrix(confusion_matrix, figsize=(10, 7), class_names=class_names)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Visualizing randomly selected images along with their predicted and true labels
selected_images = []
selected_labels = []
random_indices = random.sample(range(len(test_images)), 10)
for idx in random_indices:
    selected_images.append(test_images[idx])
    selected_labels.append(test_labels_categorical[idx])
selected_images = np.array(selected_images)
selected_labels = np.array(selected_labels)

fig = plt.figure(figsize=(20, 8))
rows = 2
cols = 5
index = 1
for image, label in zip(selected_images, selected_labels):
    fig.add_subplot(rows, cols, index)
    prediction = model.predict(tf.expand_dims(image, axis=0))
    prediction = class_names[tf.argmax(prediction.flatten())]
    label = class_names[tf.argmax(label)]
    plt.title(f"Label: {label}, Prediction: {prediction}")
    plt.imshow(image/255.)
    plt.axis("off")
    index += 1
plt.show()
